In [ ]:

import torch
import torch.nn as nn
import numpy as np
import json
#from transformers import pipeline


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# === Load class ID to word mapping ===
with open("sign_to_prediction_index_map.json", "r") as f:
    word_to_class = json.load(f)
class_to_word = {v: k for k, v in word_to_class.items()}

## === Step 1: Rebuild the model ===

# Define the Model class

# from model import Model

In [ ]:
class Model(nn.Module):
    def __init__(self,num_embed, d_model, max_len, n_heads, num_encoders, num_classes, dropout, activation, batch_first = True):
        super(Model,self).__init__()
        self.d_model = d_model
        self.max_len = max_len
        self.n_heads = n_heads
        self.num_encoders = num_encoders
        self.dropout = dropout
        self.num_embed = num_embed

        self.embed = nn.ModuleList()

        self.embed.append(nn.Linear(d_model, d_model*2, bias = True))
        self.embed.append(nn.LayerNorm(d_model*2))
        self.embed.append(torch.nn.ReLU(inplace = True))
        for i in range(num_embed-2):
            self.embed.append(torch.nn.Linear(d_model*2, d_model*2,bias = True))
            self.embed.append(nn.LayerNorm(d_model*2))
            self.embed.append(torch.nn.ReLU(inplace = True))
        self.embed.append(nn.Linear(d_model*2, d_model, bias = True))
        self.embed.append(nn.LayerNorm(d_model))
        self.embed.append(torch.nn.ReLU(inplace = True))


        self.positionalEncoder = PositionalEncoding(d_model = d_model, max_len = max_len)
        self.cls_embedding = nn.Parameter(torch.zeros((1, d_model)))
        self.encoderLayer = nn.TransformerEncoderLayer(d_model = self.d_model,
                                                       nhead = self.n_heads,
                                                       dim_feedforward=self.d_model*2,
                                                      dropout = self.dropout,
                                                      activation = activation,
                                                      batch_first = batch_first)
        self.Encoder = nn.TransformerEncoder(encoder_layer = self.encoderLayer, num_layers = self.num_encoders)
        self.output = nn.Linear(self.d_model, num_classes)


    def forward(self, x):
        for layer in self.embed:
            x = layer(x)

        x = self.positionalEncoder(x)
        x = x + self.cls_embedding
        x = self.Encoder(x)
        x = x[:, -1, :]
        x = self.output(x)

        return x



In [ ]:
class PositionalEncoding(nn.Module):

    def __init__(self, d_model: int, dropout: float = 0.1, max_len: int = 135):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)

        position = torch.arange(max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) * (-math.log(10000.0) / d_model))
        pe = torch.zeros(1, max_len, d_model)
        pe[0, :,  0::2] = torch.sin(position * div_term)
        pe[0, :,  1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = x + self.pe
        return self.dropout(x).to(device)

In [ ]:


BEST_PARAMS = {
    "num_embed_layers": 4 ,
    "n_heads": 4,
    "n_encoder_layers": 2,
    "dropout": 0.11073790254354612
}


In [ ]:
# Define device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
import math

# Create model instance
model = Model(
    num_embed=BEST_PARAMS["num_embed_layers"],
    d_model=176,  # <--- must match the saved model
    max_len=135,
    n_heads=BEST_PARAMS["n_heads"],
    num_encoders=BEST_PARAMS["n_encoder_layers"],
    num_classes=250,
    dropout=BEST_PARAMS["dropout"],
    activation='relu',
    batch_first=True
)


## === Step 2: Load the saved model weights ===

In [ ]:
state_dict = torch.load("best_model_final.pth")
filtered_state_dict = {
    k: v for k, v in state_dict.items()
    if k in model.state_dict() and model.state_dict()[k].shape == v.shape
}
model.load_state_dict(filtered_state_dict, strict=False)

model.eval()


/tmp/ipykernel_812249/15744313.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load("best_model_final.pth")


Model(
  (embed): ModuleList(
    (0): Linear(in_features=176, out_features=352, bias=True)
    (1): LayerNorm((352,), eps=1e-05, elementwise_affine=True)
    (2): ReLU(inplace=True)
    (3): Linear(in_features=352, out_features=352, bias=True)
    (4): LayerNorm((352,), eps=1e-05, elementwise_affine=True)
    (5): ReLU(inplace=True)
    (6): Linear(in_features=352, out_features=352, bias=True)
    (7): LayerNorm((352,), eps=1e-05, elementwise_affine=True)
    (8): ReLU(inplace=True)
    (9): Linear(in_features=352, out_features=176, bias=True)
    (10): LayerNorm((176,), eps=1e-05, elementwise_affine=True)
    (11): ReLU(inplace=True)
  )
  (positionalEncoder): PositionalEncoding(
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoderLayer): TransformerEncoderLayer(
    (self_attn): MultiheadAttention(
      (out_proj): NonDynamicallyQuantizableLinear(in_features=176, out_features=176, bias=True)
    )
    (linear1): Linear(in_features=176, out_features=352, bias=True)
    (dropo

## === Step 3: Prepare a single input ===

In [ ]:
# import numpy as np
X_test = np.load("X_test.npy", allow_pickle=True)

In [ ]:
print(type(X_test))
print(type(X_test[0]))  # Check the type of the first element
print(X_test[0])  # Inspect the data


<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
['train_landmark_files/28656/3311214787.parquet']


In [ ]:
ls -l

total 15984
-rw-r--r--  1 vscode vscode   23940 Apr 27 12:55  ASL_Tranformer_Inference_April25_1.ipynb
-rw-r--r--  1 vscode vscode   58060 Apr 27 15:29 'ASL_Tranformer_Inference_with Evaluation_April26_1.ipynb'
-rw-r--r--  1 vscode vscode  450517 Apr 27 15:28  X_test.npy
-rw-r--r--  1 vscode vscode 4053085 Apr 27 15:28  X_train.npy
-rw-r--r--  1 vscode vscode 5795946 Apr 27 12:54  best_model_final.pth
-rw-r--r--  1 vscode vscode 5205410 Apr 27 12:52  final_train.csv
-rw-r--r--  1 vscode vscode    3352 Apr 27 12:52  sign_to_prediction_index_map.json
drwxr-xr-x 23 vscode vscode    4096 Apr 27 12:54  train_landmark_files/
-rw-r--r--  1 vscode vscode   75712 Apr 27 15:29  y_test.npy
-rw-r--r--  1 vscode vscode  680360 Apr 27 15:29  y_train.npy


In [ ]:
import pandas as pd

# Load the parquet file for "Hello" Sign

parquet_data = pd.read_parquet('train_landmark_files/55372/1458242525.parquet')

#parquet_data = pd.read_parquet('train_landmark_files/28656/1460359.parquet')
#"C:\Users\aryama\Desktop\Aryama_data255_April21\train_landmark_files\28656\1460359.parquet"
# Check the contents of the parquet file
print(parquet_data.head())


    x0   x1   x2   x3   x4   x5   x6   x7   x8   x9  ...  y78  y79  y80  y81  \
0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0  ... -1.0 -1.0 -1.0 -1.0   
1 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0  ... -1.0 -1.0 -1.0 -1.0   
2 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0  ... -1.0 -1.0 -1.0 -1.0   
3 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0  ... -1.0 -1.0 -1.0 -1.0   
4 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0  ... -1.0 -1.0 -1.0 -1.0   

   y82  y83  y84  y85  y86  y87  
0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0  
1 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0  
2 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0  
3 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0  
4 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0  

[5 rows x 176 columns]


In [ ]:
# If the data contains numerical columns, extract them as a numpy array
data_array = parquet_data.to_numpy()

# Pick a single sample and convert to tensor
single_sample = data_array[0]  # assuming you're using the first row
single_sample_tensor = torch.tensor(single_sample, dtype=torch.float32).to(device)
single_sample_tensor = single_sample_tensor.unsqueeze(0)  # Shape: (1, 135, D)



### === Step 4: Inference ===

In [ ]:
# Assuming 'device' is already set to either 'cpu' or 'cuda'
model = model.to(device)  # Make sure the model is on the same device as the input

with torch.no_grad():
    output = model(single_sample_tensor)  # Now the model and input tensor are on the same device
    predicted_class = output.argmax(dim=1).item()

print(f"Predicted class for sample 0: {predicted_class}")

Predicted class for sample 0: 105


###  Predict Top-5 classes ===

In [ ]:
with torch.no_grad():
    output = model(single_sample_tensor)
    top5_prob, top5_labels = torch.topk(output, 5, dim=1)

print(f"Top-5 predicted classes: {top5_labels.cpu().numpy()[0]}")
print(f"Top-5 probabilities: {top5_prob.softmax(dim=1).cpu().numpy()[0]}")


Top-5 predicted classes: [105  44  80 208 175]
Top-5 probabilities: [0.3786606  0.21133378 0.1447588  0.14234644 0.12290039]


### Sentence Level Prediction

In [ ]:
## pip install transformers
#pip install git+https://github.com/huggingface/transformers
#git clone https://github.com/huggingface/transformers.git
#cd transformers
#pip install -e .

In [ ]:
# === Step 1: Load necessary components ===

# Define device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Assume Model class is already defined elsewhere
# from model import Model

# Example: best hyperparameters (update if needed)
BEST_PARAMS = {
    "num_embed_layers": 4,
    "n_heads": 4,
    "n_encoder_layers": 2,
    "dropout": 0.11
}

In [ ]:
sign_model = Model(
    num_embed=BEST_PARAMS["num_embed_layers"],
    d_model=176,
    max_len=135,
    n_heads=BEST_PARAMS["n_heads"],
    num_encoders=BEST_PARAMS["n_encoder_layers"],
    num_classes=250,
    dropout=BEST_PARAMS["dropout"],
    activation='relu',
    batch_first=True
)

In [ ]:
state_dict = torch.load("best_model_final.pth", map_location=device)
filtered_state_dict = {
    k: v for k, v in state_dict.items()
    if k in sign_model.state_dict() and sign_model.state_dict()[k].shape == v.shape
}
sign_model.load_state_dict(filtered_state_dict, strict=False)
sign_model = sign_model.to(device)
sign_model.eval()


/tmp/ipykernel_812249/3842101145.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load("best_model_final.pth", map_location=device)


Model(
  (embed): ModuleList(
    (0): Linear(in_features=176, out_features=352, bias=True)
    (1): LayerNorm((352,), eps=1e-05, elementwise_affine=True)
    (2): ReLU(inplace=True)
    (3): Linear(in_features=352, out_features=352, bias=True)
    (4): LayerNorm((352,), eps=1e-05, elementwise_affine=True)
    (5): ReLU(inplace=True)
    (6): Linear(in_features=352, out_features=352, bias=True)
    (7): LayerNorm((352,), eps=1e-05, elementwise_affine=True)
    (8): ReLU(inplace=True)
    (9): Linear(in_features=352, out_features=176, bias=True)
    (10): LayerNorm((176,), eps=1e-05, elementwise_affine=True)
    (11): ReLU(inplace=True)
  )
  (positionalEncoder): PositionalEncoding(
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoderLayer): TransformerEncoderLayer(
    (self_attn): MultiheadAttention(
      (out_proj): NonDynamicallyQuantizableLinear(in_features=176, out_features=176, bias=True)
    )
    (linear1): Linear(in_features=176, out_features=352, bias=True)
    (dropo

In [ ]:
import torch
print(torch.__version__)

2.5.1


In [ ]:
# === Step 6.1: Load test data (file paths) ===
X_train = np.load("X_train.npy", allow_pickle=True)
y_train = np.load("y_train.npy", allow_pickle=True)

In [ ]:
# === Step 7: Inference loop ===
import random
import pandas as pd
import torch

predicted_words = set()  # use a set to auto-remove duplicates
attempts = 0
max_attempts = 100  # prevent infinite loop

# === Step 7: Inference loop ===
while len(predicted_words) < 5 and attempts < max_attempts:
    idx = random.choice(range(len(X_test)))  # pick random sample
    file_path = X_test[idx][0]
    df = pd.read_parquet(file_path)

    # Correct: don't flatten
    sample_features = df.values  # shape (135, 176)
    sample_tensor = torch.tensor(sample_features, dtype=torch.float32).unsqueeze(0).to(device)  # (1, 135, 176)

    with torch.no_grad():
        output = sign_model(sample_tensor)
        predicted_class = output.argmax(dim=1).item()

    predicted_word = class_to_word.get(predicted_class, "unknown")

    if predicted_word != "unknown":
        if predicted_word not in predicted_words:
            print(f"Sample {idx}: Predicted unique word: {predicted_word}")
        predicted_words.add(predicted_word)

    attempts += 1  # avoid infinite loops if not enough unique words

# Finally, convert set to list
predicted_words = list(predicted_words)

print("\nFinal 5 unique predicted words:", predicted_words)


Sample 9166: Predicted unique word: fast
Sample 1747: Predicted unique word: milk
Sample 7219: Predicted unique word: empty
Sample 1085: Predicted unique word: aunt

Final 5 unique predicted words: ['fast', 'milk', 'empty', 'aunt']


### Correct word prediction

In [ ]:
import pandas as pd
import torch
import random

correct_predicted_results = []  # To store correct predictions only
unique_correct_words = set()
correct_predictions = 0
attempts = 0
max_attempts = 1000  # Safety cap

# === Step 7: Predict from test set until 10 unique correctly predicted words are found ===
while len(unique_correct_words) < 10 and attempts < max_attempts:
    idx = random.choice(range(len(X_test)))
    file_path = X_test[idx][0]
    df = pd.read_parquet(file_path)

    sample_features = df.values  # shape (135, 176)
    sample_tensor = torch.tensor(sample_features, dtype=torch.float32).unsqueeze(0).to(device)

    with torch.no_grad():
        output = sign_model(sample_tensor)
        predicted_class = output.argmax(dim=1).item()

    predicted_word = class_to_word.get(predicted_class, "unknown")
    ground_truth_class = y_test[idx]
    ground_truth_word = class_to_word.get(ground_truth_class, "unknown")

    is_correct = (predicted_class == ground_truth_class)

    if is_correct and predicted_word != "unknown" and predicted_word not in unique_correct_words:
        correct_predicted_results.append({
            "Sample Index": idx,
            "Predicted Word": predicted_word,
            "Ground Truth Word": ground_truth_word
        })
        unique_correct_words.add(predicted_word)
        print(f"[{len(unique_correct_words)}/10] Sample {idx}: Correctly Predicted: {predicted_word}")

    attempts += 1

# === Step 8: Summary Results ===
print(f"\nCollected {len(unique_correct_words)} unique correct predicted words after {attempts} attempts.")

# Optionally display as DataFrame
results_df = pd.DataFrame(correct_predicted_results)
print("\nSummary of correct predictions:\n", results_df)


[1/10] Sample 5640: Correctly Predicted: fast

Collected 1 unique correct predicted words after 1000 attempts.

Summary of correct predictions:
    Sample Index Predicted Word Ground Truth Word
0          5640           fast              fast


## Evaluation

In [ ]:
import pandas as pd
import torch
import random

correct_predicted_results = []  # To store correct predictions only
unique_correct_words = set()
correct_predictions = 0
attempts = 0
max_attempts = 1000  # Safety cap

# === Step 7: Predict from test set until 10 unique correctly predicted words are found ===
while len(unique_correct_words) < 10 and attempts < max_attempts:
    idx = random.choice(range(len(X_train)))
    file_path = X_train[idx][0]
    df = pd.read_parquet(file_path)

    sample_features = df.values  # shape (135, 176)
    sample_tensor = torch.tensor(sample_features, dtype=torch.float32).unsqueeze(0).to(device)

    with torch.no_grad():
        output = sign_model(sample_tensor)
        predicted_class = output.argmax(dim=1).item()

    predicted_word = class_to_word.get(predicted_class, "unknown")
    ground_truth_class = y_train[idx]
    ground_truth_word = class_to_word.get(ground_truth_class, "unknown")

    is_correct = (predicted_class == ground_truth_class)

    if is_correct and predicted_word != "unknown" and predicted_word not in unique_correct_words:
        correct_predicted_results.append({
            "Sample Index": idx,
            "Predicted Word": predicted_word,
            "Ground Truth Word": ground_truth_word
        })
        unique_correct_words.add(predicted_word)
        print(f"[{len(unique_correct_words)}/10] Sample {idx}: Correctly Predicted: {predicted_word}")

    attempts += 1

# === Step 8: Summary Results ===
print(f"\nCollected {len(unique_correct_words)} unique correct predicted words after {attempts} attempts.")

# Optionally display as DataFrame
results_df = pd.DataFrame(correct_predicted_results)
print("\nSummary of correct predictions:\n", results_df)

# Extract only the correctly predicted words
correctly_predicted_words = [entry['Predicted Word'] for entry in correct_predicted_results]


[1/10] Sample 56801: Correctly Predicted: fast

Collected 1 unique correct predicted words after 1000 attempts.

Summary of correct predictions:
    Sample Index Predicted Word Ground Truth Word
0         56801           fast              fast


## WER

In [ ]:
import pandas as pd
import torch
import random

def compute_wer(references, hypotheses):
    """
    Compute Word Error Rate (WER) for isolated sign language predictions.

    Args:
        references (list): List of ground-truth glosses (e.g., ["APPLE", "HOUSE"]).
        hypotheses (list): List of predicted glosses (e.g., ["APPLE", "TREE"]).

    Returns:
        float: WER as (substitutions + deletions + insertions) / total reference words.
    """
    if not references:
        return 0.0 if not hypotheses else float('inf')

    substitutions = 0
    deletions = 0
    insertions = 0
    total_words = len(references)

    # For isolated signs, each pair is a single gloss
    for ref, hyp in zip(references, hypotheses):
        if ref != hyp:
            substitutions += 1  # Incorrect prediction = substitution

    # Handle length mismatches
    if len(hypotheses) < len(references):
        deletions += len(references) - len(hypotheses)
    elif len(hypotheses) > len(references):
        insertions += len(hypotheses) - len(references)

    total_errors = substitutions + deletions + insertions
    return total_errors / total_words if total_words > 0 else 0.0

# Initialize lists for tracking predictions
correct_predicted_results = []  # Store correct predictions
unique_correct_words = set()    # Track unique correct glosses
all_predictions = []            # Store all predictions for WER
all_references = []            # Store all ground-truth glosses for WER
attempts = 0
max_attempts = 1000

# === Step 7: Predict until 10 unique correctly predicted words are found ===
while len(unique_correct_words) < 10 and attempts < max_attempts:
    idx = random.choice(range(len(X_train)))
    file_path = X_train[idx][0]
    df = pd.read_parquet(file_path)

    sample_features = df.values  # shape (135, 176)
    sample_tensor = torch.tensor(sample_features, dtype=torch.float32).unsqueeze(0).to(device)

    with torch.no_grad():
        output = sign_model(sample_tensor)
        predicted_class = output.argmax(dim=1).item()

    predicted_word = class_to_word.get(predicted_class, "unknown")
    ground_truth_class = y_train[idx]
    ground_truth_word = class_to_word.get(ground_truth_class, "unknown")

    # Store predictions and references for WER
    all_predictions.append(predicted_word)
    all_references.append(ground_truth_word)

    is_correct = (predicted_class == ground_truth_class)

    if is_correct and predicted_word != "unknown" and predicted_word not in unique_correct_words:
        correct_predicted_results.append({
            "Sample Index": idx,
            "Predicted Word": predicted_word,
            "Ground Truth Word": ground_truth_word
        })
        unique_correct_words.add(predicted_word)
        print(f"[{len(unique_correct_words)}/10] Sample {idx}: Correctly Predicted: {predicted_word}")

    attempts += 1

# === Step 8: Compute WER and Summarize Results ===
wer = compute_wer(all_references, all_predictions)
print(f"\nCollected {len(unique_correct_words)} unique correct predicted words after {attempts} attempts.")
print(f"Word Error Rate (WER): {wer:.4f}")

# Display results as DataFrame
results_df = pd.DataFrame(correct_predicted_results)
print("\nSummary of correct predictions:\n", results_df)

# Extract correctly predicted words
correctly_predicted_words = [entry['Predicted Word'] for entry in correct_predicted_results]

[1/10] Sample 37934: Correctly Predicted: fast
[2/10] Sample 35417: Correctly Predicted: cry

Collected 2 unique correct predicted words after 1000 attempts.
Word Error Rate (WER): 0.9960

Summary of correct predictions:
    Sample Index Predicted Word Ground Truth Word
0         37934           fast              fast
1         35417            cry               cry


## Sentence Generation with predicted words from signer

In [ ]:
predicted_words

['fast', 'milk', 'empty', 'aunt']

In [ ]:
import openai

client= openai.OpenAI(api_key= "API key")

## Case 1:

In [ ]:
def generate_sentence_with_gpt(predicted_words):
    prompt = (
        f"Create a short, creative, grammatically correct sentence "
        f"using ONLY these words: {', '.join(predicted_words)}. "
        f"Do not add any extra words. You can rearrange them, repeat if needed, "
        f"but do not introduce new words."
    )

    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a creative sentence generator."},
            {"role": "user", "content": prompt}
        ],
        temperature=0.3,
        max_tokens=20,
        n=1,
    )
    return response.choices[0].message.content

In [ ]:
#predicted_words = ['kitty', 'milk', 'if', 'fast', 'finger']
generated_sentence = generate_sentence_with_gpt(predicted_words)
print("\nGenerated Sentence:", generated_sentence)



Generated Sentence: Aunt fast emptied the milk.


## Case 2:

In [ ]:
def generate_sign_language_sentence(predicted_words):
    prompt = (
        f"You are creating a meaningful, natural-sounding sentence suitable for a sign language performance. "
        f"Use ONLY these words: {', '.join(predicted_words)}. "
        f"Make the sentence visual, imaginative, and easy to express with body language. "
        f"Do NOT introduce new words, but you can rearrange or repeat words creatively if needed. "
        f"Keep it short and vivid."
    )

    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a visual sentence crafter for sign language."},
            {"role": "user", "content": prompt}
        ],
        temperature=0.2,  # more creativity
        max_tokens=20,     # allow longer sentences
        n=1,
    )
    return response.choices[0].message.content

# Usage
generated_sentence = generate_sign_language_sentence(predicted_words)
print("\nGenerated Sign Language Sentence:", generated_sentence)



Generated Sign Language Sentence: Aunt fast empty milk.


In [ ]:
generated_sentence1 = generate_sign_language_sentence(predicted_words)
print("\nGenerated Sentence:", generated_sentence1)


Generated Sentence: Aunt fast empty milk.


## Case 1 : Bleu Score

In [ ]:
# Rearranging manually to desired ASL order
asl_ordered_words = ['Aunt', 'fast','empty','milk']
asl_gloss_sentence = " ".join(asl_ordered_words)
reference_sentence=asl_gloss_sentence

In [ ]:
asl_gloss_sentence

'Aunt fast empty milk'

In [ ]:
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction

def compute_bleu_score(reference_sentence, generated_sentence):
    """
    Compute BLEU score between reference and generated sentence.
    """
    reference_tokens = reference_sentence.lower().split()
    candidate_tokens = generated_sentence.lower().split()

    smoothie = SmoothingFunction().method4  # smoothing helps with short sentences

    score = sentence_bleu(
        [reference_tokens],
        candidate_tokens,
        smoothing_function=smoothie,
        weights=(0.5, 0.5)  # consider unigrams and bigrams equally
    )
    return score

# Example Usage:
#reference_sentence = "The owl cut the puzzle pieces, alligator did the same."
generated_sentence = generate_sentence_with_gpt(predicted_words)

bleu = compute_bleu_score(reference_sentence, generated_sentence)
print("\nGenerated Sentence:", generated_sentence)
print(f"BLEU Score: {bleu:.4f}")



Generated Sentence: Fast Aunt emptied the milk.
BLEU Score: 0.1269


## Case 2 : Bleu Score:

In [ ]:
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction

def compute_bleu_score(reference_sentence, generated_sentence):
    """
    Compute BLEU score between reference and generated sentence.
    """
    reference_tokens = reference_sentence.lower().split()
    candidate_tokens = generated_sentence.lower().split()

    smoothie = SmoothingFunction().method4  # smoothing helps with short sentences

    score = sentence_bleu(
        [reference_tokens],
        candidate_tokens,
        smoothing_function=smoothie,
        weights=(0.5, 0.5)  # consider unigrams and bigrams equally
    )
    return score

# Example Usage:
#reference_sentence = "The owl cut the puzzle pieces, alligator did the same."
generated_sentence = generate_sign_language_sentence(predicted_words)

bleu = compute_bleu_score(reference_sentence, generated_sentence)
print("\nGenerated Sentence:", generated_sentence)
print(f"BLEU Score: {bleu:.4f}")


Generated Sentence: Aunt fast empty milk.
BLEU Score: 0.7071


## Sentence Formation with correctly Predicted words /GLOSS:

In [ ]:
correctly_predicted_words

['cut', 'alligator', 'dog']

In [ ]:
# Rearranging manually to desired ASL order
asl_ordered_words = ['alligator', 'cut', 'dog']

In [ ]:
asl_gloss_sentence = " ".join(asl_ordered_words)

In [ ]:
print("Rearranged ASL Gloss Words:", asl_ordered_words)
print("ASL Gloss Sentence:", asl_gloss_sentence)

Rearranged ASL Gloss Words: ['alligator', 'cut', 'dog']
ASL Gloss Sentence: alligator cut dog


In [ ]:
generated_sentence1 = generate_sentence_with_gpt( asl_ordered_words)
print("\nGenerated Sentence:", generated_sentence1)


Generated Sentence: The dog cut the alligator.


In [ ]:
generated_sentence2 = generate_sign_language_sentence( asl_ordered_words)
print("\nGenerated Sentence:", generated_sentence2)



Generated Sentence: The dog cut the alligator.


In [ ]:
reference_sentence = asl_gloss_sentence


bleu = compute_bleu_score(reference_sentence, generated_sentence1)
print("\nGenerated Sentence:", generated_sentence1)
print(f"BLEU Score: {bleu:.4f}")



Generated Sentence: The dog cut the alligator.
BLEU Score: 0.1269


In [ ]:
from jiwer import wer

In [ ]:
predicted_words = asl_ordered_words
ground_truth_words =  asl_ordered_words

# Convert to space-separated sentences
#predicted_sentence = " ".join(predicted_words)
ground_truth_sentence = " ".join(ground_truth_words)

# WER calculation
error_rate = wer(ground_truth_sentence, generated_sentence1)

print(f"Predicted: {generated_sentence1}")
print(f"Ground Truth: {ground_truth_sentence}")
print(f"WER: {error_rate:.4f}")


Predicted: alligator cut dog
Ground Truth: alligator cut dog
WER: 1.3333
